# Построение и оптимизация модели

В этом задании вам предстоит поучаствовать в соревновании на [kaggle inclass](https://inclass.kaggle.com/c/telecom-clients-churn-prediction).
В соревновании вы будете работать с той же выборкой, что и ранее, поэтому воспользуйтесь результатами, полученными на предыдущих неделях. Для успешного участия в соревновании необходимо преодолеть по качеству beseline решение.

Итак, мы научились обрабатывать данные, выбрали схему кросс-валидации и определились с метриками качества. Пора переходить к оптимизации модели. На этой неделе вам предстоит принять участие в соревновании на платформе kaggle inclass! Цель такого соревнования - преодолеть предложенное baseline решение, а, главное, обсудить и сравнить предложенные решения на форуме. Какие признаки оказали наибольший вклад в модель? Как лучше обрабатывать категориальные признаки? Нужно ли делать отбор признаков, А балансировать выборку? Экспериментируйте с данными и обсуждайте ваши решения на форуме!

In [1]:
import pandas as pd
import random
import numpy as np
from scipy.sparse import coo_matrix, hstack
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from sklearn.metrics import roc_curve, precision_recall_curve, f1_score, roc_auc_score, recall_score, precision_score, log_loss

from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

seed = 1903
first_categorial_index = 190

Загрузим train dataset.

In [2]:
churn_data_frame = pd.read_csv("..\..\Data\churn_data_train.csv", ",")
churn_labels_frame = pd.read_csv("..\..\Data\churn_labels_train.csv")
kaggle_data_frame = pd.read_csv("..\..\Data\orange_small_kaggle_test_data.csv", ",")
print(churn_data_frame.shape)
print(churn_labels_frame.shape)

(27999, 230)
(27999, 1)


## Общая предобработка признаков

In [3]:
def fill_numericna(train_frame, test_frame, averageCalculator):
    """ Функция заполняет значения в числовом фрейме значениями, посчитанными averageCalculator. """
    
    # Посчитаем средние по колонкам
    numeric_avgs = averageCalculator(train_frame)
    
    # Оставим только те колонки, в которых среднее значение не равно NaN, т.к. в таких колонках совсем нет значений
    numeric_avgs = numeric_avgs.dropna()
    dropped_columns = train_frame.columns.drop(numeric_avgs.index)
    n_frame_train = train_frame[list(numeric_avgs.index)]
    n_frame_test = test_frame[list(numeric_avgs.index)]
    
    # Заполним пропущенные численные значения средними
    n_frame_train = n_frame_train.fillna(numeric_avgs, axis=0)
    n_frame_test = n_frame_test.fillna(numeric_avgs, axis=0)
    return (n_frame_train, n_frame_test, dropped_columns)

def fill_numericna_means(train_frame, test_frame):
    """ Функция заполняет значения в числовом фрейме средними и удаляет те колонки, в которых значений нет. """
    return fill_numericna(
        train_frame,
        test_frame,
        lambda f: f.mean(axis=0, skipna=True))

def fill_numericna_medians(train_frame, test_frame):
    """ Функция заполняет значения в числовом фрейме медианами и удаляет те колонки, в которых значений нет. """
    return fill_numericna(
        train_frame,
        test_frame,
        lambda f: f.median(axis=0, skipna=True))
    
def remove_constant_features(frame, min_count=2):
    """Функция удаляет колонки, которые содержат только одно значение."""
    
    # Посчитаем количества уникальных значений по колонкам
    unique_counts = frame.nunique()
    # Удалим колонки с количеством значений меньшим min_count
    columns_to_drop = unique_counts[unique_counts < min_count].index
    
    return (frame.drop(columns=columns_to_drop), columns_to_drop)

def fill_na_frequent_values(frame):
    """ Функция заполняет пустые значения самым частым значением в колонке. """
    result = frame.copy()
    for column in result.columns:
        frequencies = result[column].value_counts()
        if (len(frequencies) < 1):
            continue
        most_frequent_value = frequencies.index[0]
        result[column] = result[column].fillna(most_frequent_value)
    return result

class MatrixLabelEncoder:
    """ Класс кодирует категории числами от 0 до n, где n количество категорий в колонке. """
    
    def __init__(self):
        self.encoders = []
    
    def fit(self, matrix):
        for column_number in range(matrix.shape[1]):
            column = matrix[:,column_number]
            labelEncoder = LabelEncoder().fit(column)
            self.encoders.append(labelEncoder)
        return self
    
    def transform(self, matrix):
        transformed = np.empty(matrix.shape)
        for column_number in range(matrix.shape[1]):
            labelEncoder = self.encoders[column_number]
            num_column = labelEncoder.transform(matrix[:,column_number])
            for row_number, val in enumerate(num_column):
                transformed[row_number, column_number] = val
        return transformed
    
class CompositeEncoder:
    def __init__(self, encoder_factories):
        self.encoder_factories = encoder_factories
        
    def fit(self, matrix):
        encoders = []
        transformed = matrix
        for encoder_factory in self.encoder_factories:
            encoder = encoder_factory().fit(transformed)
            encoders.append(encoder)
            transformed = encoder.transform(transformed)
        self.encoders = encoders
        return self

    def transform(self, matrix):
        for encoder in self.encoders:
            matrix = encoder.transform(matrix)
        return matrix

Выделим числовые и категориальные признаки.

In [4]:
numeric_columns = churn_data_frame.columns[:first_categorial_index]
categorial_columns = churn_data_frame.columns[first_categorial_index:]

In [5]:
def predict_ridge_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn модель Ridge """
    # Поскольку RidgeClassifier не обладает функцией predict_proba приходится считать его вручную
    # Подробнее можно посмотреть здесь:
    # https://www.codesd.com/item/scikit-learn-ridge-classifier-extract-class-probabilities.html
    func = model.decision_function(X)
    return np.exp(func) / (1 + np.exp(func))

def predict_model_proba(X, model):
    """ Функция возвращает вероятности предсказаний для класса churn """
    return list(zip(*model.predict_proba(X)))[1]

def get_model_data(model_with_data):
    """ Функция принимает на вход результат stratifiedKFold_fscore и возвращает только модель и её X_test и y_test """
    model = model_with_data[1]
    split = model_with_data[3]
    X = split[2]
    y = split[3]
    return (model,X,y)

def calculate_metrics(model_with_data, predict_probabilities):
    """ Посчитаем метрики качества для модели """
    model, X, y = get_model_data(model_with_data)
    predictions = model.predict(X)
    probabilities = predict_probabilities(X, model)
    # Считаем F-меру, precision и recall
    fscore = f1_score(y, predictions)
    precision = precision_score(y, predictions)
    recall = recall_score(y, predictions)
    # Считаем Log loss
    logLoss = log_loss(y, probabilities)
    # Считаем roc auc score
    rocAuc = roc_auc_score(y, probabilities)
    return (precision,recall,fscore,logLoss,rocAuc)

def transform_to_chart_model(label, model_with_data, predict_probabilities):
    """ Функция принимает на вход заголовок, результат функции stratifiedKFold_fscore и функцию, считающую вероятности
    и возвращает заголовок, вектор ответов и вектор вероятностей для построения графиков """
    model, X, y = get_model_data(model_with_data)
    probabilities = predict_probabilities(X, model)
    return (label, y, probabilities)

def charts_row(model_charts, chart_builder, figsize=(11, 4)):
    """ Функция принимает данные для построения графиков и логику построения одного графика и строит графики в одну строку """
    fig, axes = plt.subplots(
        nrows=1,
        ncols=len(model_charts),
        figsize=figsize,
        sharey=True)
    for i, chart_data in enumerate(model_charts):
        ax = axes[i]
        chart_builder(chart_data, ax, i)
    plt.show()
    
def scatter(chart_data, ax, ax_index, T=0.5):
    """ Функция строит график распределения вероятностей по классам """
    label, actual, predicted = chart_data
    ax.scatter(actual, predicted)
    ax.set_xlabel("Labels")
    if ax_index == 0:
        ax.set_ylabel("Predicted probabilities")
    ax.set_title(label)
    ax.plot([-1.1, 1.2], [T, T])
    ax.axis([-1.1, 1.1, -0.1, 1.1])
    
def precision_recal_thresh(chart_data, ax, ax_index):
    """ Функция строит графики значений precision и recall в зависимости от порога """
    label, actual, predicted = chart_data
    prec, rec, thresh = precision_recall_curve(actual, predicted)
    min_len = min([len(prec),len(rec),len(thresh)])
    ax.plot(thresh[:min_len], prec[:min_len], label="precision")
    ax.plot(thresh[:min_len], rec[:min_len], label="recall")
    ax.legend(loc=1)
    ax.set_xlabel("threshold")
    ax.set_title(label)
    
def auc_prc(chart_data, ax, ax_index):
    """ Функция строит график AUC PRC (зависимость precision от recall)"""
    label, actual, predicted = chart_data
    prec, rec, thresh = precision_recall_curve(actual, predicted)
    min_len = min([len(prec),len(rec),len(thresh)])
    ax.plot(rec[:min_len], prec[:min_len])
    ax.set_xlabel("recall")
    if ax_index == 0:
        ax.set_ylabel("precision")
    ax.set_title(label)

def calc_min_distance (actual,predicted):
    """ Функция считает минимальное расстояние до точки [0;1] для AUC ROC """
    fpr,tpr,thr = roc_curve(actual,predicted)
    distance,fpr_v,tpr_v,thr_v = min(zip(np.sqrt((1.-tpr)**2+fpr**2),fpr,tpr,thr), key=lambda d:d[0])
    return (distance,fpr_v,tpr_v,thr_v)
    
def auc_roc(chart_data, ax, ax_index):
    """ Функция строит кривую AUC ROC и отмечает точку, соответствующую минимальному расстоянию до точки [0;1] """
    label, actual, predicted = chart_data
    fpr, tpr, thr = roc_curve(actual, predicted)
    min_dist,min_fpr,min_tpr,_ = calc_min_distance(actual, predicted)
    ax.plot(fpr, tpr, label="ROC AUC curve")
    ax.scatter(min_fpr,min_tpr,color="red")
    ax.set_xlabel("false positive rate")
    if ax_index == 0:
        ax.set_ylabel("true positive rate")
    ax.legend(loc=4)
    ax.set_title(label)

In [6]:
def stratifiedKFold_fscore(
    frame,
    labels,
    model_factory,
    process_frame,
    frame_to_matrix,
    numeric_features,
    categorial_features,
    predict_probabilities,
    seed,
    folds_count = 3):
    """ Функция разбивает набор данных на folds_count, считает ROC-AUC на каждом фолде
        и возвращает усредненное по фолдам значение.
        Функция также возвращает модель, показавшую лучшее качество, её метрики и разделение данных.
        Разделение данных нужно для того, чтобы строить метрики модели на данных, на которых она не обучалась."""
    skf = StratifiedKFold(n_splits=folds_count, shuffle=True, random_state=seed)
    
    best_model = None
    best_score = 0
    best_precision = 0
    best_recall = 0
    best_table = None
    best_split = None
    best_encoders = None
    best_dropped_columns = None
    metrics_sum = 0
    for train_indices, test_indices in skf.split(frame, labels):
        # Разобьем фрем на train и test с помощью функции process_frame
        # Внутри такой функции мы можем по-разному обрабатывать признаки обучаясь только на train наборе.
        train_frame, train_labels, test_frame, test_labels, dropped_numeric, dropped_categorial = process_frame(
            frame.loc[train_indices, :],
            labels.loc[train_indices, :],
            frame.loc[test_indices, :],
            labels.loc[test_indices, :],
            numeric_features,
            categorial_features)
        numeric_cleaned = numeric_features.drop(dropped_numeric)
        categorial_cleaned = categorial_features.drop(dropped_categorial)
        # Преобразуем фреймы в матрицы.
        # Тут можно выполнить финальное преобразование признаков, например масштабирование признаков.
        # В функции frame_to_matrix энкодеры типа StandardScaler обучаются только на train признаках.
        X_train, X_test, num_encoder, cat_encoder = frame_to_matrix(
            train_frame,
            test_frame,
            numeric_cleaned,
            categorial_cleaned)
        y_train = train_labels.as_matrix().flatten()
        y_test = test_labels.as_matrix().flatten()

        model = model_factory()
        # Обучим модель
        model.fit(X_train, y_train)
        
        # Построим вероятности принадлежности к целевому классу
        probabilities = predict_probabilities(X_test, model)
        # Считаем roc auc score
        rocAuc = roc_auc_score(y_test, probabilities)
        metrics_sum += rocAuc
        if(best_model is None or best_score < rocAuc):
            # В случае, если модель лучше предыдущих сохраним её
            # оценку, модель, матрицу ошибок и разделение данных
            best_score = rocAuc
            best_model = model
            best_dropped_columns = (dropped_numeric, dropped_categorial)
            best_encoders = (num_encoder, cat_encoder)
            best_split = (X_train, y_train, X_test, y_test)
    return (
        metrics_sum/folds_count,
        best_model,
        best_score,
        best_split,
        best_encoders,
        best_dropped_columns)

In [7]:
def cleanup_frame_common(frame, numeric_features, categorial_features):
    """Функция делит признакина числовые и категориальные и удаляет константные признаки, содержащие только одно значение"""
    # Разделим коллекции на группы - числовые и категориальные.
    numeric_frame = frame[numeric_features].copy()
    categorial_frame = frame[categorial_features].copy()
    # Удалим вещественные колонки, содержащие одно и менее значений. 0 значений мы получаем, когда значения во всех строках Nan.
    numeric_frame_no_const, dropped_const_numeric_columns = remove_constant_features(numeric_frame)
    
    # Удалим категориальные колонки, содержащие ноль значений. Если есть одно значение, то могут быть Nan, которые для
    # категориальных признаков могут быть еще одной категорией (зависит от стратегии обработки).
    categorial_frame_no_const, dropped_const_categorial_columns = remove_constant_features(categorial_frame, 1)
    
    # Восстановим фрейм и вернем вместе с ним список удаленных категориальных колонок.
    return (pd.concat([numeric_frame, categorial_frame], axis=1),
            list(dropped_const_numeric_columns),
            list(dropped_const_categorial_columns))

def process_frame_base_model(train_frame, train_labels, test_frame, test_labels, numeric_features, categorial_features):
    """ Функция строит базовую модель из предыдущей недели """
    
    # Удалим константные колонки из train_frame, и такие-же колонки из test_frame
    train_frame, const_numeric_columns, const_categorial_columns = cleanup_frame_common(
        train_frame,
        numeric_features,
        categorial_features)
    test_frame = test_frame.drop(columns=const_numeric_columns)
    test_frame = test_frame.drop(columns=const_categorial_columns)
    
    numeric_features = numeric_features.drop(const_numeric_columns)
    categorial_features = categorial_features.drop(const_categorial_columns)
    
    # Заполним пропущенные вещественные значения средними
    numeric_train, numeric_test, dropped_numeric = fill_numericna_means(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    numeric_features = numeric_features.drop(dropped_numeric)
    
    # Заполним пропущенные категориальные значения строками "NaV" (Not a value)
    categorial_train = train_frame[categorial_features].fillna("NaV")
    categorial_test = test_frame[categorial_features].fillna("NaV")
    
    # Удалим категориальные колонки с одним единственным значением
    categorial_train, dropped_categorial = remove_constant_features(categorial_train)
    categorial_test = categorial_test.drop(columns=dropped_categorial)
    
    categorial_features = categorial_features.drop(dropped_categorial)
    
    # Список удаленных колонок
    dropped_numeric = np.concatenate([
        list(const_numeric_columns),
        list(dropped_numeric)])
    dropped_categorial = np.concatenate([
        list(const_categorial_columns),
        list(dropped_categorial)])
    
    return (pd.concat([numeric_train, categorial_train], axis=1),
            train_labels,
            pd.concat([numeric_test, categorial_test], axis=1),
            test_labels,
            dropped_numeric,
            dropped_categorial)

def scale_features(train_frame, test_frame):
    train_numeric = train_frame.as_matrix()
    
    scaler = StandardScaler().fit(train_numeric)
    
    train_numeric = coo_matrix(scaler.transform(train_numeric))
    test_numeric = coo_matrix(scaler.transform(test_frame.as_matrix()))
    
    return (train_numeric, test_numeric, scaler)

def one_hot_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    
    if fit_matrix.shape[0] == 0 or fit_matrix.shape[1] == 0:
        return (coo_matrix(train_frame.as_matrix()), coo_matrix(test_frame.as_matrix()), None)
    categorial_encoder = CompositeEncoder([MatrixLabelEncoder, OneHotEncoder]).fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def int_label_features(train_frame, test_frame):
    fit_matrix = pd.concat([train_frame, test_frame]).as_matrix()
    categorial_encoder = MatrixLabelEncoder().fit(fit_matrix)
    
    train_categorial = categorial_encoder.transform(train_frame.as_matrix())
    test_categorial = categorial_encoder.transform(test_frame.as_matrix())
    
    return (train_categorial, test_categorial, categorial_encoder)

def frame_to_matrix_one_hot(train_frame, test_frame, numeric_features, categorial_features):
    """ Функци преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные с помощью OneHotEncoding. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    # One hot encode для категориальных признаков
    train_categorial, test_categorial, categorial_encoder = one_hot_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            scaler,
            categorial_encoder)

def frame_to_matrix_labeled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные целыми числами. """
    
    # Масштабируем вещественные признаки
    train_numeric, test_numeric, scaler = scale_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            scaler,
            categorial_encoder)

def ridge_one_hot_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        process_frame_base_model,
        frame_to_matrix_one_hot, 
        numeric_columns,
        categorial_columns,
        predict_ridge_proba,
        seed)

def random_forest_labeled_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        process_frame_base_model,
        frame_to_matrix_labeled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

def gradient_boosting_labeled_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        process_frame_base_model,
        frame_to_matrix_labeled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

Базовые модели, построенные на предыдущей неделе

In [8]:
ridge_one_hot_base = ridge_one_hot_builder(churn_data_frame, churn_labels_frame)
random_forest_base = random_forest_labeled_builder(churn_data_frame, churn_labels_frame)
gradient_boosting_base = gradient_boosting_labeled_builder(churn_data_frame, churn_labels_frame)

## Sampling
Проверим увеличит ли сэмплирование данных качество модели

In [9]:
def balance_frame_sampling(frame_to_balance, labels, repeat_times):
    """ Функция балансирует соотношения классов сэмплируя класс churn. """
    y_name = "labels"
    churn_only_labels = labels[labels[y_name] == 1]
    churn_indices = list(churn_only_labels.index)
    churn_only_frame = frame_to_balance.loc[churn_indices,:]
    churn_balanced_frame = frame_to_balance.copy()
    churn_balanced_labels = labels.copy()
    for i in range(0, (repeat_times-1)):
        churn_balanced_frame = pd.concat([churn_balanced_frame, churn_only_frame], ignore_index=True)
        churn_balanced_labels = pd.concat([churn_balanced_labels, churn_only_labels], ignore_index=True)
        
    churn_balanced_frame[y_name] = churn_balanced_labels[y_name]
    churn_balanced_frame = churn_balanced_frame.sample(frac=1).reset_index(drop=True)
    
    churn_balanced_labels = pd.DataFrame(churn_balanced_frame[y_name], columns=[y_name])
    churn_balanced_frame = churn_balanced_frame.drop(columns=[y_name])
    
    return (churn_balanced_frame, churn_balanced_labels)

def balance_train_frame(train_frame, train_labels, test_frame, test_labels, numeric_features, categorial_features, repeat_times):
    """ Функция балансирует train frame, после этого обрабатывает признаки с помощью метода
        process_frame_base_model """
    balanced_train, balanced_labels = balance_frame_sampling(train_frame, train_labels, repeat_times)
    return process_frame_base_model(balanced_train, balanced_labels, test_frame, test_labels, numeric_features, categorial_features)

def balanced_ridge_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе RidgeClassifier, предварительно балансируя выборки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_one_hot,
        numeric_features,
        categorial_features,
        predict_ridge_proba,
        seed)

def balanced_random_forest_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе RandomForestClassifier, предварительно балансируя выборки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

def balanced_gradient_boosting_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе GradientBoostingClassifier, предварительно балансируя выборки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_labeled,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

Построим RidgeClassifier, RandomForestClassifier и GradientBoostingClassifier и сравним результат с базовым решением

In [10]:
repeat_times = 12
ridge_balanced = balanced_ridge_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns,
    repeat_times)
random_forest_balanced = balanced_random_forest_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns,
    repeat_times)
gradient_boosting_balanced = balanced_gradient_boosting_builder(
    churn_data_frame,
    churn_labels_frame,
    numeric_columns,
    categorial_columns,
    repeat_times)

In [11]:
indices = ["Ridge Base", "Ridge Balanced", "Random Forest Base", "Random Forest Balanced", "Gradient boosting base", "Gradient boosting balanced"]
print(pd.DataFrame(
    [
        [ridge_one_hot_base[0], ridge_one_hot_base[2]],
        [ridge_balanced[0], ridge_balanced[2]],
        [random_forest_base[0], random_forest_base[2]],
        [random_forest_balanced[0], random_forest_balanced[2]],
        [gradient_boosting_base[0], gradient_boosting_base[2]],
        [gradient_boosting_balanced[0], gradient_boosting_balanced[2]]
    ],
    index=indices,
    columns=["Mean", "Best"]))
print()
print(pd.DataFrame(
    [
        calculate_metrics(ridge_one_hot_base, predict_ridge_proba),
        calculate_metrics(ridge_balanced, predict_ridge_proba),
        calculate_metrics(random_forest_base, predict_model_proba),
        calculate_metrics(random_forest_balanced, predict_model_proba),
        calculate_metrics(gradient_boosting_base, predict_model_proba),
        calculate_metrics(gradient_boosting_balanced, predict_model_proba),
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

                                Mean      Best
Ridge Base                  0.675126  0.686025
Ridge Balanced              0.671609  0.683472
Random Forest Base          0.588253  0.600968
Random Forest Balanced      0.610274  0.614934
Gradient boosting base      0.732486  0.742115
Gradient boosting balanced  0.723209  0.734313

                            Precision    Recall   F-Score  Log Loss   Roc Auc
Ridge Base                   0.000000  0.000000  0.000000  0.414959  0.686025
Ridge Balanced               0.130529  0.522302  0.208861  0.611811  0.683472
Random Forest Base           0.333333  0.004323  0.008535  1.017398  0.600968
Random Forest Balanced       0.280000  0.010072  0.019444  0.809283  0.614934
Gradient boosting base       0.187500  0.004323  0.008451  0.240476  0.742115
Gradient boosting balanced   0.143142  0.621037  0.232659  0.556841  0.734313


Из таблицы выше видно, что при балансировке классов стабильно растет Recall и F-Score. Roc Auc при этом просел аж для двух моделей (Ridge и Gradient Boosting).
Log Loss для Gradient Boosting при балансироваке также вырос.
Судя по тому, что по большей части метрики растут и растут сильно (в десятках процентов) я в данном ноутбуке буду балансировать классы.

## Масштабировать или нет

Попробуем не масшатабировать модель. Сравним результаты и решим, что лучше, - масштабировать признаки или нет.

In [9]:
def to_matrix_one_hot_not_scaled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функци преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные с помощью OneHotEncoding. """
    
    train_numeric = coo_matrix(train_frame[numeric_features].as_matrix())
    test_numeric = coo_matrix(test_frame[numeric_features].as_matrix())
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    # One hot encode для категориальных признаков
    train_categorial, test_categorial, categorial_encoder = one_hot_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            None,
            categorial_encoder)

def to_matrix_labeled_not_scaled(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки и кодирует категориальные целыми числами. """
    
    train_numeric = coo_matrix(train_frame[numeric_features].as_matrix())
    test_numeric = coo_matrix(test_frame[numeric_features].as_matrix())
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            None,
            categorial_encoder)

In [10]:
def not_scaled_bal_ridge_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, 12),
        to_matrix_one_hot_not_scaled,
        numeric_columns,
        categorial_columns,
        predict_ridge_proba,
        seed)

def not_scaled_bal_random_forest_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, 12),
        to_matrix_labeled_not_scaled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

def not_scaled_bal_gradient_boosting_builder(frame, labels):
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, 12),
        to_matrix_labeled_not_scaled,
        numeric_columns,
        categorial_columns,
        predict_model_proba,
        seed)

In [16]:
ridge_not_scaled = not_scaled_bal_ridge_builder(churn_data_frame, churn_labels_frame)
random_forest_not_scaled = not_scaled_bal_random_forest_builder(churn_data_frame, churn_labels_frame)
gradient_boosting_not_scaled = not_scaled_bal_gradient_boosting_builder(churn_data_frame, churn_labels_frame)

In [27]:
indices = ["Ridge Balanced", "Ridge Not Scaled", "Random Forest Balanced",
           "Random Forest Not Scaled", "Gradient boosting balanced", "Gradient boosting Not Scaled"]
print(pd.DataFrame(
    [
        [ridge_balanced[0], ridge_balanced[2]],
        [ridge_not_scaled[0], ridge_not_scaled[2]],
        [random_forest_balanced[0], random_forest_balanced[2]],
        [random_forest_not_scaled[0], random_forest_not_scaled[2]],
        [gradient_boosting_balanced[0], gradient_boosting_balanced[2]],
        [gradient_boosting_not_scaled[0], gradient_boosting_not_scaled[2]]
    ],
    index=indices,
    columns=["Mean", "Best"]))
print()
print(pd.DataFrame(
    [
        calculate_metrics(ridge_balanced, predict_ridge_proba),
        calculate_metrics(ridge_not_scaled, predict_ridge_proba),
        calculate_metrics(random_forest_balanced, predict_model_proba),
        calculate_metrics(random_forest_not_scaled, predict_model_proba),
        calculate_metrics(gradient_boosting_balanced, predict_model_proba),
        calculate_metrics(gradient_boosting_not_scaled, predict_model_proba),
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

                                  Mean      Best
Ridge Balanced                0.204751  0.208801
Ridge Not Scaled              0.152178  0.156084
Random Forest Balanced        0.021115  0.024862
Random Forest Not Scaled      0.015695  0.019499
Gradient boosting balanced    0.232492  0.234680
Gradient boosting Not Scaled  0.232407  0.234615

                              Precision    Recall   F-Score  Log Loss  \
Ridge Balanced                 0.130482  0.522302  0.208801  0.611813   
Ridge Not Scaled               0.090996  0.548201  0.156084  0.679135   
Random Forest Balanced         0.300000  0.012968  0.024862  0.842268   
Random Forest Not Scaled       0.304348  0.010072  0.019499  0.905647   
Gradient boosting balanced     0.145041  0.614388  0.234680  0.548544   
Gradient boosting Not Scaled   0.144992  0.614388  0.234615  0.548541   

                               Roc Auc  
Ridge Balanced                0.683467  
Ridge Not Scaled              0.574823  
Random Forest Balance

В основном метрики либо не растут, либо уменьшаются. На мой взгляд лучше масштабировать признаки

## Полиномиальные фичи
Перед отбором признаков добавим к числовым признакам их квадраты и попарные произведения, тем самым перейдем в спрямляющее пространство.


In [11]:
def add_polynomial_features(train_frame, test_frame):
    train_numeric = train_frame.as_matrix()
    
    numeric_encoder = CompositeEncoder([StandardScaler, PolynomialFeatures]).fit(train_numeric)
    
    train_numeric = coo_matrix(numeric_encoder.transform(train_numeric))
    test_numeric = coo_matrix(numeric_encoder.transform(test_frame.as_matrix()))
    
    return (train_numeric, test_numeric, numeric_encoder)

def frame_to_matrix_one_hot_poly(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки, добавляет их квадраты и попарные произведения.
        Кодирует категориальные признаки с помощью OneHotEncoding. """
    
    # Масштабируем вещественные признаки и добавим к ним их квадраты и попарные произведения
    train_numeric, test_numeric, numeric_encoder = add_polynomial_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    # One hot encode для категориальных признаков
    train_categorial, test_categorial, categorial_encoder = one_hot_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            numeric_encoder,
            categorial_encoder)

def frame_to_matrix_labeled_poly(train_frame, test_frame, numeric_features, categorial_features):
    """ Функция преобразует фрейм к sparse матрице.
        Масштабирует вещественные признаки, добавляет их квадраты и попарные произведения.
        Кодирует категориальные признаки целыми числами. """
    
    # Масштабируем вещественные признаки и добавим к ним их квадраты и попарные произведения
    train_numeric, test_numeric, numeric_encoder = add_polynomial_features(
        train_frame[numeric_features],
        test_frame[numeric_features])
    
    # Закодируем категориальные признаки значениями от 0 до n с помощью MatrixLabelEncoder
    train_categorial, test_categorial, categorial_encoder = int_label_features(
        train_frame[categorial_features],
        test_frame[categorial_features])
    
    return (hstack([train_numeric, train_categorial]),
            hstack([test_numeric, test_categorial]),
            numeric_encoder,
            categorial_encoder)

In [12]:
def poly_ridge_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе RidgeClassifier, добавляя полиномиальные признаки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        RidgeClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_one_hot_poly,
        numeric_features,
        categorial_features,
        predict_ridge_proba,
        seed)

def poly_random_forest_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе RandomForestClassifier, добавляя полиномиальные признаки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        RandomForestClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_labeled_poly,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

def poly_gradient_boosting_builder(frame, labels, numeric_features, categorial_features, repeat_times):
    """ Функция строит модель на основе GradientBoostingClassifier, добавляя полиномиальные признаки."""
    return stratifiedKFold_fscore(
        frame,
        labels,
        GradientBoostingClassifier,
        lambda tf, tl, tsf, tsl, nf, cf: balance_train_frame(tf, tl, tsf, tsl, nf, cf, repeat_times),
        frame_to_matrix_labeled_poly,
        numeric_features,
        categorial_features,
        predict_model_proba,
        seed)

In [23]:
ridge_poly = poly_ridge_builder(
    churn_data_frame,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)
random_forest_poly = poly_random_forest_builder(
    churn_data_frame,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)
gradient_boosting_poly = poly_gradient_boosting_builder(
    churn_data_frame,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)

In [ ]:
indices = ["Ridge Balanced","Ridge Poly", "Random Forest Balanced", "Random Forest Poly", "Gradient boosting balanced", "Gradient boosting Poly"]
print(pd.DataFrame(
    [
        calculate_metrics(ridge_balanced, predict_ridge_proba),
        calculate_metrics(ridge_poly, predict_ridge_proba),
        calculate_metrics(random_forest_balanced, predict_model_proba),
        calculate_metrics(random_forest_poly, predict_model_proba),
        calculate_metrics(gradient_boosting_balanced, predict_model_proba),
        calculate_metrics(gradient_boosting_poly, predict_model_proba)
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

## Отбор признаков
Строим GradientBoostingClassifier, добавляя по одному вещественные признаки, которые повышают качество.

In [13]:
repeat_times=12
num_cols_selected = pd.Index([])
best_metric = 0
best_model = None
best_numeric_features = pd.Index([])

for col in numeric_columns:
    num_cols_selected = num_cols_selected.insert(len(num_cols_selected), col)
    gb_selected_num = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        num_cols_selected,
        pd.Index([]),
        repeat_times)
    
    auc_roc = gb_selected_num[2]
    if np.isin(col, gb_selected_num[5][0], invert=True) and np.round(auc_roc, 5) > np.round(best_metric, 5):
        best_metric = auc_roc
        best_model = gb_selected_num
        best_numeric_features = num_cols_selected.copy()
    else:
        num_cols_selected = num_cols_selected.drop(col)
        
print (best_metric, best_numeric_features)

0.724014191032 Index(['Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var13',
       'Var14', 'Var16', 'Var17', 'Var25', 'Var27', 'Var28', 'Var29', 'Var34',
       'Var49', 'Var51', 'Var53', 'Var58', 'Var61', 'Var73', 'Var74', 'Var81',
       'Var82', 'Var83', 'Var88', 'Var90', 'Var99', 'Var110', 'Var113',
       'Var116', 'Var126', 'Var127', 'Var140', 'Var142', 'Var147', 'Var157',
       'Var161', 'Var172', 'Var186', 'Var187', 'Var189', 'Var190'],
      dtype='object')


Также по-одному добавим категориальные признаки

In [14]:
cat_cols_selected = pd.Index([])
best_cat_metric = 0
best_cat_model = None
best_cat_features = pd.Index([])

for col in categorial_columns:
    cat_cols_selected = cat_cols_selected.insert(len(cat_cols_selected), col)
    gb_selected_cat = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        best_numeric_features,
        cat_cols_selected,
        repeat_times)
    
    auc_roc = gb_selected_cat[2]
    if np.isin(col, gb_selected_cat[5][1], invert=True) and np.round(auc_roc, 5) > np.round(best_cat_metric, 5):
        best_cat_metric = auc_roc
        best_cat_model = gb_selected_cat
        best_cat_features = cat_cols_selected.copy()
    else:
        cat_cols_selected = cat_cols_selected.drop(col)
        
print (best_cat_metric, best_cat_features)

0.739212100143 Index(['Var191', 'Var193', 'Var194', 'Var196', 'Var201', 'Var205', 'Var208',
       'Var211', 'Var212', 'Var213', 'Var215', 'Var217', 'Var218', 'Var220',
       'Var221', 'Var229'],
      dtype='object')


Строим GradientBoostingClassifier с отобранными категориальными и вещественными признаками, удаляем те вещественные признаки, которые уменьшают качество.

In [15]:
num_cols_selected = best_numeric_features.copy()
very_best_metric = best_metric
very_best_model = best_model
very_best_num_features = best_numeric_features.copy()

for col in best_numeric_features:
    num_cols_selected = num_cols_selected.drop(col)
    gb_selected_num = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        num_cols_selected,
        best_cat_features,
        repeat_times)
    
    auc_roc = gb_selected_num[2]
    if np.round(auc_roc, 5) >= np.round(very_best_metric, 5):
        very_best_metric = auc_roc
        very_best_model = gb_selected_num
        very_best_num_features = num_cols_selected.copy()
    else:
        num_cols_selected = num_cols_selected.insert(len(num_cols_selected), col)
        
print (very_best_metric, very_best_num_features)

0.741112422624 Index(['Var90', 'Var99', 'Var110', 'Var113', 'Var116', 'Var126', 'Var127',
       'Var140', 'Var142', 'Var147', 'Var157', 'Var161', 'Var172', 'Var186',
       'Var187', 'Var189', 'Var190', 'Var3', 'Var4', 'Var5', 'Var7', 'Var13',
       'Var14', 'Var16', 'Var17', 'Var25', 'Var28', 'Var29', 'Var34', 'Var49',
       'Var51', 'Var53', 'Var58', 'Var61', 'Var73', 'Var81', 'Var82', 'Var83'],
      dtype='object')


Строим модель, удаляя те категориальные признаки, которые понижают качество

In [16]:
cat_cols_selected = best_cat_features.copy()
very_best_cat_metric = very_best_metric
very_best_cat_model = very_best_model
very_best_cat_features = best_cat_features.copy()

for col in best_cat_features:
    cat_cols_selected = cat_cols_selected.drop(col)
    gb_selected_cat = balanced_gradient_boosting_builder(
        churn_data_frame,
        churn_labels_frame,
        very_best_num_features,
        cat_cols_selected,
        repeat_times)
    auc_roc = gb_selected_cat[2]
    if np.round(auc_roc, 5) >= np.round(very_best_cat_metric, 5):
        very_best_cat_metric = auc_roc
        very_best_cat_model = gb_selected_cat
        very_best_cat_features = cat_cols_selected.copy()
    else:
        cat_cols_selected = cat_cols_selected.insert(len(cat_cols_selected), col)
        
print (very_best_cat_metric, very_best_cat_features)

0.741840723884 Index(['Var221', 'Var229', 'Var191', 'Var193', 'Var194', 'Var196', 'Var201',
       'Var205', 'Var208', 'Var211', 'Var212', 'Var213', 'Var215', 'Var217',
       'Var218'],
      dtype='object')


In [17]:
calculate_metrics(very_best_cat_model, predict_model_proba)

(0.140625,
 0.63544668587896258,
 0.23028720626631854,
 0.56711936525031048,
 0.74184072388407762)

Обратим внимание, что в kaggle датасете есть пара колонок (Var209 и Var230) в которых совсем нет данных. Удалим их, т.к. они не сыграют роли при построении вероятностей для kaggle

In [24]:
#columns_contain_lots_new_cats = [
#    "Var200", "Var214", "Var217", "Var199", "Var198",
#    "Var220", "Var222", "Var202", "Var216"]
#columns_not_contain_data = ["Var209", "Var230"]
#num_columns_to_remove = ["Var20", "Var79"]
#numeric_columns_cleaned = very_best_num_features.drop(num_columns_to_remove)
#categorial_columns_no_new_cats = very_best_cat_features #.drop(columns_not_contain_data)
all_best_features = pd.Index(list(very_best_num_features) + list(very_best_cat_features))
frame_best_features = churn_data_frame[all_best_features]

Построим ridge, random forest и gradient boosting классификаторы основываясь на отборе признаков проведенном выше

In [25]:
ridge_add_del = balanced_ridge_builder(
    frame_best_features,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)
random_forest_add_del = balanced_random_forest_builder(
    frame_best_features,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)
gradient_boosting_add_del = balanced_gradient_boosting_builder(
    frame_best_features,
    churn_labels_frame,
    very_best_num_features,
    very_best_cat_features,
    repeat_times)

In [28]:
indices = ["Ridge poly", "Ridge add del","Random forest poly", "Random forest add del",
           "GB Poly", "GB Add del"]
print(pd.DataFrame(
    [
        [ridge_poly[0], ridge_poly[2]],
        [ridge_add_del[0], ridge_add_del[2]],
        [random_forest_poly[0], random_forest_poly[2]],
        [random_forest_add_del[0], random_forest_add_del[2]],
        [gradient_boosting_poly[0], gradient_boosting_poly[2]],
        [gradient_boosting_add_del[0], gradient_boosting_add_del[2]]
    ],
    index=indices,
    columns=["Mean", "Best"]))
print()
print(pd.DataFrame(
    [
        calculate_metrics(ridge_poly, predict_ridge_proba),
        calculate_metrics(ridge_add_del, predict_ridge_proba),
        calculate_metrics(random_forest_poly, predict_model_proba),
        calculate_metrics(random_forest_add_del, predict_model_proba),
        calculate_metrics(gradient_boosting_poly, predict_model_proba),
        calculate_metrics(gradient_boosting_add_del, predict_model_proba)
    ],
    index=indices,
    columns=["Precision", "Recall", "F-Score", "Log Loss", "Roc Auc"]))

                           Mean      Best
Ridge poly             0.655119  0.657426
Ridge add del          0.674195  0.685113
Random forest poly     0.636211  0.642317
Random forest add del  0.639326  0.645680
GB Poly                0.721617  0.729056
GB Add del             0.727653  0.741635

                       Precision    Recall   F-Score  Log Loss   Roc Auc
Ridge poly              0.160870  0.106628  0.128250  0.672250  0.657426
Ridge add del           0.136850  0.515850  0.216314  0.615196  0.685113
Random forest poly      0.133333  0.017266  0.030573  0.827347  0.642317
Random forest add del   0.186667  0.020144  0.036364  0.840164  0.645680
GB Poly                 0.139930  0.629683  0.228976  0.561926  0.729056
GB Add del              0.140625  0.635447  0.230287  0.567213  0.741635


Видно, что на удалось чуть-чуть улучшить модели за счет отбора

## Построение решения для kaggle

Ячейка ниже использовалась для поиска новых категорий в test dataset и поиска пустых колонок.

In [18]:
# Посмотреть насколько больше категорий стало
cols_to_remove = []
print(kaggle_data_frame.shape[0])
for col in very_best_cat_features:
    full_train_col = set(churn_data_frame[col].as_matrix())
    kaggle_col = set(kaggle_data_frame[col].as_matrix())
    intersected = (kaggle_col & full_train_col)
    for v in intersected:
        kaggle_col.remove(v)
    cols_to_remove.append((col, len(kaggle_col)))
print (list(ctr[0] for ctr in sorted(cols_to_remove, key=lambda ctr: -ctr[1]) if ctr[1] > 1))

10000
['Var217', 'Var212']


Заменим новые категории (которых очень много) самыми частыми по колонкам

In [19]:
cols_to_drop = []
cleaned_kaggle_data_frame = kaggle_data_frame.copy()
for col in very_best_cat_features:
    column = cleaned_kaggle_data_frame[col]
    train_column = churn_data_frame[col]
    full_train_col = set(train_column.as_matrix())
    kaggle_col = set(column.as_matrix())
    intersected = list(kaggle_col & full_train_col)
    for v in intersected:
        kaggle_col.remove(v)
    frequencies = train_column.value_counts()
    if (len (frequencies) < 1):
        cols_to_drop.append(col)
        continue
    most_frequent_value = frequencies.index[0]
    for to_remove in kaggle_col:
        column = column.replace(to_remove, most_frequent_value)
    cleaned_kaggle_data_frame[col] = column
cols_to_drop

[]

Применим теже преобразования, какие мы применяли для обучения тренировочной модели к kaggle датасету.

In [20]:
# very_best_cat_model very_best_cat_features very_best_num_features
kaggle_numeric = cleaned_kaggle_data_frame[very_best_num_features].copy()
kaggle_categorial = cleaned_kaggle_data_frame[very_best_cat_features].copy()

numeric_encoder, categorial_encoder = very_best_cat_model[4]
k_t_n, kaggle_num_means, kaggle_dropped_columns = fill_numericna_means(kaggle_numeric, kaggle_numeric)
kaggle_means_scaled = numeric_encoder.transform(kaggle_num_means)

kaggle_frame_nav = fill_na_frequent_values(kaggle_categorial)
kaggle_encoded = categorial_encoder.transform(kaggle_frame_nav.as_matrix())

X_kaggle = hstack([
    coo_matrix(kaggle_means_scaled),
    kaggle_encoded
])

Получим вероятности для Kaggle и сохраним их в отдельный файл

In [21]:
kg_predict_probabilities = predict_ridge_proba(X_kaggle, very_best_cat_model[1])
kg_predict_frame = pd.DataFrame(kg_predict_probabilities, columns=["result"])
kg_predict_frame.head()

,result
0,0.315521
1,0.533128
2,0.246412
3,0.293048
4,0.166113


In [22]:
kg_predict_frame.to_csv("..\..\Results\churn_kaggle_gb_add_del.csv", index=True, index_label="Id")

## Заключение
В этой модели мы применили
- label encoding и one-hot encoding для обработки категориальных признаков
- scaling для обработки вещественных признаков
- сэмплирование для балансировки классов
- отбор признаков по методу add-del

Нам надо было сделать решение, которое превосходило бы базовое решение на kaggle.
На kaggle есть два лидер борда (private и public) и там чуть-чуть разные оценки
Private: 0.68981
Public: 0.65318

Моя модель на kaggle показала следующие результаты (см. скриншот)
Private: 0.71212
Public: 0.66762

Таким образом можно утверждать, что я улучшил базовое решение.

P.S. К сожалению соревнование уже закончено и я так и не смог попасть в LeaderBoard, но я приложил все скриншоты, на которых можно посмотреть мою оценку на kaggle.

In [ ]:
# Заполнение вещественных пропусков (нулями, медианами)
# Заполнение категориальных пропусков (частотами)
# Полиномиальные фичи